In [8]:
import requests
from requests import Response

youbike_url = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

res:Response = requests.get(youbike_url)
if res.status_code == 200:
    print("下載成功")
else:
    print("下載失敗")


from pydantic import BaseModel, Field, RootModel, field_validator

class Site(BaseModel):
    站名:str = Field(alias = 'sna')
    所在區:str = Field(alias='sarea')
    位置:str = Field(alias='ar')
    日期:str = Field(alias='mday')
    狀態:bool = Field(alias = "act")
    總車輛數:int = Field(alias='total')
    可借:int = Field(alias='available_rent_bikes')
    可還:int = Field(alias='available_return_bikes')
    經度:float = Field(alias='longitude')
    緯度:float = Field(alias='latitude')

# 將驗證器附加到模型的特定欄位  若要先進行驗證再輸出 用 mode="before" 

    @field_validator("站名")
    @classmethod
    def abc(cls,value:str)->str:
        return value.split("_")[-1]

class YouBike(RootModel):
    root: list[Site]

data:YouBike = YouBike.model_validate_json(res.text)

all_sites:list = data.model_dump()
all_sites

import pandas as pd
df1 = pd.DataFrame(all_sites)
df1

下載成功


,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
0,捷運科技大樓站,大安區,復興南路二段235號前,2024-06-18 15:56:22,True,28,0,28,121.54360,25.02605
1,復興南路二段273號前,大安區,復興南路二段273號西側,2024-06-18 15:56:22,True,21,2,19,121.54357,25.02565
2,國北教大實小東側門,大安區,和平東路二段96巷7號,2024-06-18 15:56:22,True,16,16,0,121.54124,25.02429
3,和平公園東側,大安區,和平東路二段118巷33號,2024-06-18 15:57:20,True,11,9,2,121.54282,25.02351
4,辛亥復興路口西北側,大安區,復興南路二段368號,2024-06-18 15:43:21,True,16,12,4,121.54299,25.02153
...,...,...,...,...,...,...,...,...,...,...
1422,臺大總圖書館西南側,臺大公館校區,臺大圖書館西南側,2024-06-18 15:52:15,True,30,20,0,121.54031,25.01690
1423,臺大黑森林西側,臺大公館校區,臺大霖澤館南側,2024-06-18 15:56:22,True,20,15,5,121.54347,25.01995
1424,臺大獸醫館南側,臺大公館校區,臺大獸醫系館南側,2024-06-18 15:57:19,True,24,14,6,121.54242,25.01791
1425,臺大新體育館東南側,臺大公館校區,臺大體育館東側,2024-06-18 15:56:22,True,40,39,1,121.53591,25.02112


In [9]:
df1['可借'] <= 3

0        True
1        True
2       False
3       False
4       False
        ...  
1422    False
1423    False
1424    False
1425    False
1426    False
Name: 可借, Length: 1427, dtype: bool

pandas.DataFrame.info
- DataFrame.info(verbose=None, buf=None, max_cols=None, memory_usage=None, show_counts=None)[source]
- Print a concise summary of a DataFrame.

In [12]:
#檢查  null(空的)
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1427 entries, 0 to 1426
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   站名      1427 non-null   object 
 1   所在區     1427 non-null   object 
 2   位置      1427 non-null   object 
 3   日期      1427 non-null   object 
 4   狀態      1427 non-null   bool   
 5   總車輛數    1427 non-null   int64  
 6   可借      1427 non-null   int64  
 7   可還      1427 non-null   int64  
 8   經度      1427 non-null   float64
 9   緯度      1427 non-null   float64
dtypes: bool(1), float64(2), int64(3), object(4)
memory usage: 101.9+ KB


In [10]:
# 資料篩選
df1[df1['可借'] <= 3]

,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
0,捷運科技大樓站,大安區,復興南路二段235號前,2024-06-18 15:56:22,True,28,0,28,121.54360,25.02605
1,復興南路二段273號前,大安區,復興南路二段273號西側,2024-06-18 15:56:22,True,21,2,19,121.54357,25.02565
15,溫州公園,大安區,羅斯福路三段283巷11號,2024-06-18 15:56:22,True,5,3,2,121.53156,25.01895
17,溫州停車場,大安區,新生南路三段54巷/溫州街口,2024-06-03 14:08:52,False,5,0,5,121.53310,25.02093
18,銘傳國小側門,大安區,羅斯福路四段21號旁天橋下,2024-06-18 15:52:15,True,7,1,6,121.53549,25.01385
...,...,...,...,...,...,...,...,...,...,...
1384,臺大大一女舍北側,臺大公館校區,臺大女一舍北側,2024-06-18 15:52:21,True,53,2,51,121.53436,25.01547
1395,臺大共同教室北側,臺大公館校區,臺大共同教學館東北側,2024-06-18 12:43:15,True,42,0,42,121.53791,25.01595
1406,臺大椰林小舖,臺大公館校區,臺大椰林小舖旁,2024-06-18 15:54:22,True,48,1,47,121.53583,25.01561
1414,臺大一號館,臺大公館校區,臺大植物魚類標本館一號館北側,2024-06-18 15:49:19,True,19,1,18,121.53455,25.01698


pandas.DataFrame.query
- DataFrame.query(expr, *, inplace=False, **kwargs)[source]
- Query the columns of a DataFrame with a boolean expression.

pandas.DataFrame.shape
- property DataFrame.shape[source]
- Return a tuple representing the dimensionality of the DataFrame.

In [31]:
#檢查 query(全是字串)
import numpy as np
less3_df = df1.query('可借<=3')
less3_df.shape

(333, 10)

In [30]:
# and/ or 皆可用
df2 = df1.query('可借<=3 and 總車輛數<=10')
df2

,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
15,溫州公園,大安區,羅斯福路三段283巷11號,2024-06-18 15:56:22,True,5,3,2,121.53156,25.01895
17,溫州停車場,大安區,新生南路三段54巷/溫州街口,2024-06-03 14:08:52,False,5,0,5,121.53310,25.02093
18,銘傳國小側門,大安區,羅斯福路四段21號旁天橋下,2024-06-18 15:52:15,True,7,1,6,121.53549,25.01385
553,仁愛金山路口,中正區,仁愛路二段31號前,2024-06-18 15:57:20,True,9,2,7,121.52801,25.03840
676,植福樂群路口,中山區,植福路/樂群三路(西北側),2024-06-18 15:47:19,True,10,3,7,121.56040,25.08303
781,新明路343號,內湖區,新明路341號北側,2024-06-18 15:27:16,True,10,1,9,121.58345,25.05653
1263,福德國小公車站,信義區,福德街308號對側,2024-06-18 14:44:20,True,10,3,7,121.58755,25.03929
1360,萬大路493巷,萬華區,萬大路493巷44號旁,2024-06-18 15:45:21,True,10,2,8,121.49920,25.01975
1415,臺大小小福西南側,臺大公館校區,臺大進修教育大樓,2024-06-18 14:13:15,True,10,0,10,121.53686,25.01539


In [29]:
# 外層''  內層要"" 不同的
df3 = df1.query('所在區=="大安區" and 可借==0')
df3

,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
0,捷運科技大樓站,大安區,復興南路二段235號前,2024-06-18 15:56:22,True,28,0,28,121.54360,25.02605
17,溫州停車場,大安區,新生南路三段54巷/溫州街口,2024-06-03 14:08:52,False,5,0,5,121.53310,25.02093
58,捷運古亭站(5號出口),大安區,和平東路一段19號前,2024-06-18 15:56:16,True,14,0,14,121.52293,25.02715
95,捷運忠孝新生站(4號出口),大安區,忠孝新生路口北側,2024-06-18 15:49:15,True,34,0,33,121.53324,25.04239
98,瑠公公園,大安區,忠孝東路四段49巷4弄/大安路口,2024-03-30 08:59:14,False,20,0,20,121.54612,25.04201
104,捷運忠孝復興站(5號出口),大安區,復興南路一段135巷3號(對面),2024-01-14 04:19:10,False,41,0,41,121.54443,25.04262
126,忠孝東路三段217巷口,大安區,忠孝東路三段229-1號前,2024-06-06 15:05:16,False,34,0,34,121.54027,25.04176
138,市民敦化路口,大安區,敦化南路一段111號前,2024-06-18 15:56:16,True,15,0,15,121.54944,25.04464
171,安和路二段69巷口,大安區,安和路二段/安和路二段69巷,2024-06-18 15:57:20,True,20,0,20,121.55252,25.03056


In [32]:
n = 0   #代入變數
df4 = df1.query('所在區=="大安區" and 可借==@n')
df4

,站名,所在區,位置,日期,狀態,總車輛數,可借,可還,經度,緯度
0,捷運科技大樓站,大安區,復興南路二段235號前,2024-06-18 15:56:22,True,28,0,28,121.54360,25.02605
17,溫州停車場,大安區,新生南路三段54巷/溫州街口,2024-06-03 14:08:52,False,5,0,5,121.53310,25.02093
58,捷運古亭站(5號出口),大安區,和平東路一段19號前,2024-06-18 15:56:16,True,14,0,14,121.52293,25.02715
95,捷運忠孝新生站(4號出口),大安區,忠孝新生路口北側,2024-06-18 15:49:15,True,34,0,33,121.53324,25.04239
98,瑠公公園,大安區,忠孝東路四段49巷4弄/大安路口,2024-03-30 08:59:14,False,20,0,20,121.54612,25.04201
104,捷運忠孝復興站(5號出口),大安區,復興南路一段135巷3號(對面),2024-01-14 04:19:10,False,41,0,41,121.54443,25.04262
126,忠孝東路三段217巷口,大安區,忠孝東路三段229-1號前,2024-06-06 15:05:16,False,34,0,34,121.54027,25.04176
138,市民敦化路口,大安區,敦化南路一段111號前,2024-06-18 15:56:16,True,15,0,15,121.54944,25.04464
171,安和路二段69巷口,大安區,安和路二段/安和路二段69巷,2024-06-18 15:57:20,True,20,0,20,121.55252,25.03056
